<a href="https://colab.research.google.com/github/guiPython/Visualiza-o-de-Dados-UFABC-2023-/blob/main/Pratica_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação do GeoPandas

In [1]:
pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.3 MB/s eta 0:00:00


Conexão com a fonte de dados


In [62]:
import pandas as pd
import io
from google.colab import drive
drive.mount('/content/drive')


routes_git_url = "https://raw.githubusercontent.com/PaixaoThales/geovis/main/routes.txt"
airports_git_url = "https://raw.githubusercontent.com/PaixaoThales/geovis/main/airport.txt"

routes_data = pd.read_csv(routes_git_url, sep=",", encoding = "UTF-8")
airports_data = pd.read_csv(airports_git_url, sep=",", encoding = "UTF-8")
airport_traffic_data = pd.read_excel("/content/drive/MyDrive/VIS_Pratica_07/Airport_Traffic.xlsx")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Construindo Base de Dados

In [63]:
aiport_all_data = pd.DataFrame()
aiport_all_data["airport"]  = airports_data["name"]
aiport_all_data["city"] = airports_data["city"]
aiport_all_data["country"] = airports_data["country"]
aiport_all_data["longitude"] = airports_data["longitude"]
aiport_all_data["latitude"] = airports_data["latitude"]
aiport_all_data["icao"] = airports_data["ICAO"]
aiport_all_data["iata"] = airports_data["IATA"]

source_aiport_moviments = airport_traffic_data[airport_traffic_data["YEAR"] == 2023].groupby(["APT_ICAO"]).agg(source_aiport_moviments=("FLT_TOT_1","sum"))
source_aiport_moviments = source_aiport_moviments.reset_index()
source_aiport_moviments

aiport_all_data = aiport_all_data.merge(source_aiport_moviments, how="inner", left_on=["icao"], right_on=["APT_ICAO"])
del aiport_all_data["APT_ICAO"]

In [64]:
from logging import disable
from numpy.ma.core import minimum
import folium as fo
import geopandas as gp
import ipywidgets as wget
import geopy.distance

geometry = gp.points_from_xy(aiport_all_data.longitude, aiport_all_data.latitude)
geo = gp.GeoDataFrame(aiport_all_data[["airport", "country", "latitude", "longitude", "iata"]], geometry=geometry)
world = gp.read_file(gp.datasets.get_path("naturalearth_lowres"))
airports_iata_with_routes = list(set(routes_data.source_airport))
airports_with_routes = aiport_all_data[aiport_all_data["iata"].isin(airports_iata_with_routes)]
airports = list(zip(airports_with_routes.airport, airports_with_routes.iata))

def extract_iata(data, airport) -> str:
  return data[data["airport"] == airport].iata

def exists_route(data, source, final) -> bool:
  data = routes_data.loc[(routes_data['airline'] == source) & (routes_data['source_airport'] == final)]
  return not data.empty

def extract_longitude(data, source, destination) -> list:
  source_longitude = data[data["iata"] == source].longitude.values[0]
  destination_longitude = data[data["iata"] == destination].longitude.values[0]
  return {"source": source_longitude, "destination": destination_longitude}

def extract_latitude(data, source, destination) -> list:
  source_latitude = data[data["iata"] == source].latitude.values[0]
  destination_latitude = data[data["iata"] == destination].latitude.values[0]
  return {"source": source_latitude, "destination": destination_latitude}

def calculate_route_cost(source_total_moviments, source_coord, destination_coord) -> float:
  distance_between_airports = geopy.distance.geodesic(source_coord, destination_coord).km
  return round(source_total_moviments / distance_between_airports,2)

def popup_for(airport) -> str:
  name = airport.airport.values[0]
  icao = airport.icao.values[0]
  iata = airport.iata.values[0]
  country = airport.country.values[0]
  city = airport.city.values[0]
  return f"Name:{name}\nICAO:{icao}\nIATA:{iata}\nCountry:{country}\nCity:{city}"

def showing (source=None, destination=None):
  if source is not None and destination is not None:
    source_airport = aiport_all_data[aiport_all_data["iata"] == source]
    destination_airport = aiport_all_data[aiport_all_data["iata"] == destination]

    longitudes = extract_longitude(aiport_all_data, source, destination)
    latitudes = extract_latitude(aiport_all_data, source, destination)

    source_airport_coords = [latitudes["source"], longitudes["source"]]
    destination_airport_coords = [latitudes["destination"], longitudes["destination"]]

    map = fo.Map()

    fo.Marker(
        source_airport_coords, 
        popup=popup_for(source_airport),
        tooltip=source_airport.airport.values[0]).add_to(map)
    
    fo.Marker(
        destination_airport_coords, 
        popup=popup_for(destination_airport),
        tooltip=destination_airport.airport.values[0]).add_to(map)

    fo.PolyLine(
        [source_airport_coords, destination_airport_coords],
        popup=
        f"""
        Cost factor:{calculate_route_cost(
            source_airport.source_aiport_moviments.values[0], 
            source_airport_coords, 
            destination_airport_coords
        )}
        """, 
        tooltip="ROUTE").add_to(map)

    sw = [min(latitudes.values()), min(longitudes.values())]
    ne = [max(latitudes.values()), max(longitudes.values())]

    map.fit_bounds([sw, ne])
    display(map)


sources = wget.Dropdown(options=airports, value=None)
destinations = wget.Dropdown(options=airports, value=None, disabled=True)
def valid_final_airports(*args):
  if sources.value != None:
    destinations_iata = routes_data[routes_data["source_airport"] == sources.value]["airline"].unique()
    destinations_airport = aiport_all_data[aiport_all_data["iata"].isin(destinations_iata)]["airport"].unique()
    destinations.options = list(zip(destinations_airport, destinations_iata))
    destinations.value = None
    destinations.disabled = False
sources.observe(valid_final_airports, 'value')

wget.interact(showing, source=sources, destination=destinations)

interactive(children=(Dropdown(description='source', options=(('Antwerp International Airport (Deurne)', 'ANR'…

<function __main__.showing(source=None, destination=None)>

Questão 2


In [68]:
brazilian_airports = airports_data[airports_data['country'] == 'Brazil']
brazilian_airports_with_routes = brazilian_airports[brazilian_airports.IATA.isin(airports_iata_with_routes)]

def iata_destinations_of(source_iata: str) -> list:
  destinations = routes_data[routes_data["source_airport"] == source_iata]
  return list(set(destinations["airline"]))

map = fo.Map()
latitudes = []
longitudes = []

for _, source_airport in brazilian_airports_with_routes.iterrows():
  source_airport_coords = [source_airport["latitude"], source_airport["longitude"]]
  fo.Marker(source_airport_coords).add_to(map)

  for iata_destination in iata_destinations_of(source_airport["IATA"]):
    destination_airport = brazilian_airports_with_routes[brazilian_airports_with_routes["IATA"] == iata_destination]
    if not destination_airport.empty:
      destination_airport_coords = [destination_airport["latitude"].values[0], destination_airport["longitude"].values[0]]
      fo.Marker(destination_airport_coords).add_to(map)
      """fo.PolyLine(
        [source_airport_coords, destination_airport_coords]).add_to(map)"""
      
      latitudes.append(destination_airport_coords[0])
      longitudes.append(destination_airport_coords[1])
  latitudes.append(source_airport_coords[0])
  longitudes.append(source_airport_coords[1])

sw = [min(latitudes), min(longitudes)]
ne = [max(latitudes), max(longitudes)]

map.fit_bounds([sw, ne])
display(map)